In [1]:
!pip install mujoco mujoco-python-viewer pyvirtualdisplay opencv-python mediapy
!sudo apt-get install xvfb
!pip install xvfbwrapper

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/RL/code') # "RL" 폴더 이름 수정할 것

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Parse `Snapbot`

In [1]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt
from mujoco_parser import MuJoCoParserClass
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.7]


### Parse `scene_snapbot.xml`

In [2]:
xml_path = '../asset/snapbot/scene_snapbot.xml'
env = MuJoCoParserClass(name='Snapbot',rel_xml_path=xml_path,VERBOSE=True)

dt:[0.0020] HZ:[500]
n_dof (=nv):[24]
n_geom:[24]
geom_names:['floor', 'body', 'camera_module_1', 'camera_module_2', 'leg_module_1_1', 'leg_module_1_2', 'leg_module_1_3', 'leg_module_1_4', 'leg_module_1_4bar', 'leg_module_2_1', 'leg_module_2_2', 'leg_module_2_3', 'leg_module_2_4', 'leg_module_2_4bar', 'leg_module_4_1', 'leg_module_4_2', 'leg_module_4_3', 'leg_module_4_4', 'leg_module_4_4bar', 'leg_module_5_1', 'leg_module_5_2', 'leg_module_5_3', 'leg_module_5_4', 'leg_module_5_4bar']
n_body:[24]
body_names:['world', 'torso', 'Camera_module_1', 'Camera_module_2', 'Leg_module_1_1', 'Leg_module_1_2', 'Leg_module_1_3', 'Leg_module_1_4', 'Leg_module_1_4bar', 'Leg_module_2_1', 'Leg_module_2_2', 'Leg_module_2_3', 'Leg_module_2_4', 'Leg_module_2_4bar', 'Leg_module_4_1', 'Leg_module_4_2', 'Leg_module_4_3', 'Leg_module_4_4', 'Leg_module_4_4bar', 'Leg_module_5_1', 'Leg_module_5_2', 'Leg_module_5_3', 'Leg_module_5_4', 'Leg_module_5_4bar']
n_joint:[19]
joint_names:['free', 'camera_1', 'camera_2', '

### Loop

In [4]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()

video = []

env.init_viewer(viewer_title='Snapbot',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=152,distance=1.5,elevation=-30,lookat=[0.0,0.0,0.],
                  VIS_TRANSPARENT=False)
env.reset()
while env.tick < 5000:
    # Step
    env.step()
    if env.loop_every(HZ=10):
        rgb_img, _ = env.grab_rgb_depth_img()
        video.append(rgb_img)
        # Plot configuration
        PLOT_WORLD_FRAME = False
        PLOT_BODY_FRAME  = False
        PLOT_REV_JOINT   = True
        PLOT_CONTACT     = False
        # Plot world frame
        if PLOT_WORLD_FRAME:
            env.plot_T(p=np.zeros(3),R=np.eye(3,3),PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
            sim_info = 'tick:[%d] time:[%.2f]sec'%(env.tick,env.get_sim_time())
            env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),PLOT_AXIS=False,label=sim_info)
        # Plot body frames
        if PLOT_BODY_FRAME:
            for body_name in env.body_names:
                p,R = env.get_pR_body(body_name=body_name)
                env.plot_T(p=p,R=R,PLOT_AXIS=True,axis_len=0.1,axis_width=0.005)
        # Plot revolute joints with arrow
        if PLOT_REV_JOINT:
            for rev_joint_idx,rev_joint_name in zip(env.rev_joint_idxs,env.rev_joint_names):
                axis_joint = env.model.jnt_axis[rev_joint_idx]
                p_joint,R_joint = env.get_pR_joint(joint_name=rev_joint_name)
                axis_world = R_joint@axis_joint
                axis_rgba = np.append(np.eye(3)[:,np.argmax(axis_joint)],0.5)
                axis_len,axis_r = 0.05,0.005
                env.plot_arrow_fr2to(
                    p_fr=p_joint,p_to=p_joint+axis_len*axis_world,
                    r=axis_r,rgba=axis_rgba)
        # Plot contact information
        if PLOT_CONTACT:
            env.plot_contact_info(h_arrow=0.3,rgba_arrow=[1,0,1,1],PRINT_CONTACT_BODY=False)
        # Render
        env.render()

# Close viewer
env.close_viewer()


In [6]:
import cv2, os

shape = video[0].shape
video_folder = ("../video")
if not os.path.isdir(video_folder): os.mkdir(video_folder)
video_out = cv2.VideoWriter(os.path.join(video_folder, "snapbot_parse.mp4"), cv2.VideoWriter_fourcc('F','M','P','4'), 20, (shape[1], shape[0]), True)
for i in range(len(video)):
  video_frame = cv2.cvtColor(video[i], cv2.COLOR_BGR2RGB)
  video_out.write(video_frame)
video_out.release()

OpenCV: FFMPEG: tag 0x34504d46/'FMP4' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [ ]:
import mediapy
video_path = os.path.join(video_folder, "snapbot_parse.mp4")
mediapy.show_video(mediapy.read_video(video_path))
